In [1]:
import sqlparse
from sqlparse.sql import Token, TokenList
import re

In [2]:
query = '''--asd
            with a as (select a_id, 
            case when aa = \'a1\' then 1 else 0 end aa_ind
            from `a123.aaa`) 
        , b as (select b_id, 
            sum(bb) bb_sum 
            from `b123.bbb` 
            group by b_id) 
        select a.a_id,a.a_ind,b.b_sum 
        from a left join b 
        on a.a_id = b.b_id'''

In [3]:
parsed_query = sqlparse.parse(query)[0]

In [4]:
def clean_query(tokens):
    tokens_flagged = []
    for a in tokens:
        for pattern in ['Token.Text.Whitespace*','Token.Punctuation*']:
            if re.match(pattern,str(a.ttype)):
                tokens_flagged.append(a)
    tokens_cleaned = [ a for a in tokens if a not in tokens_flagged]
    tokens_cleaned = [a for a in tokens_cleaned if type(a) not in [sqlparse.sql.Comment]]
    return tokens_cleaned

In [5]:
level1_tokens = parsed_query.tokens
level1_tokens_cleaned = clean_query(level1_tokens)

In [6]:
level1_tokens_cleaned

[<CTE 'with' at 0x210C7223468>,
 <IdentifierList 'a as (...' at 0x210C72325E8>,
 <DML 'select' at 0x210C722DEE8>,
 <IdentifierList 'a.a_id...' at 0x210C7232660>,
 <Keyword 'from' at 0x210C722E468>,
 <Identifier 'a' at 0x210C7232228>,
 <Keyword 'left j...' at 0x210C722E5E8>,
 <Identifier 'b' at 0x210C72322A0>,
 <Keyword 'on' at 0x210C722E768>,
 <Comparison 'a.a_id...' at 0x210C7232390>]

In [7]:
if str(level1_tokens_cleaned[0])=='with':
    with_subq = level1_tokens_cleaned[1]


In [8]:
with_subq_tokens_cleaned = clean_query(with_subq.tokens)
with_subq_tokens_cleaned

[<Identifier 'a as (...' at 0x210C7217C00>,
 <Identifier 'b as (...' at 0x210C7232048>]

In [9]:
select_q = clean_query(with_subq_tokens_cleaned[0].tokens[-1].tokens[1:-1])

In [10]:
select_q

[<DML 'select' at 0x210C7223708>,
 <IdentifierList 'a_id, ...' at 0x210C7232408>,
 <Keyword 'from' at 0x210C722D1C8>,
 <Identifier '`a123....' at 0x210C7217DE0>]

In [10]:
q1 = '''select sum(a) asum, min(b) bmin
from (aid, max(a1) a from aaa group by aid) aa left join bb on aa.aid = bb.bid
where aa.aid!=2
group by aa.aid order by aa.aid'''
select_q1 = clean_query(sqlparse.parse(q1)[0].tokens)

In [11]:
def parse_select_q(select_q):
    select_q = clean_query(select_q)
    if str(select_q[0].ttype)=='Token.Keyword.DML' and select_q[0].value.lower()=='select':
        columns_identifier_list = select_q[1]
        print(select_q[2:])
    else:
        print("Passed query not a select query")
    #get cols dict mapping old columns to new columns
    cols_dict = {}
    for tok in columns_identifier_list.tokens:
        if type(tok)==sqlparse.sql.Identifier:
            var_formula = tok.value
            var_newname = tok.get_name()
#             if var_newname!=var_formula:
#                  var_formula = var_formula.replace(var_newname,"").strip()
            cols_dict[var_newname] = var_formula
    
    #get source table
#     if select_q[3].value=='from':
        
    
    return(cols_dict)  
        

In [12]:

parse_select_q(select_q1)

[<Keyword 'from' at 0x2202E255228>, <Identifier '(aid, ...' at 0x2202E25A750>, <Keyword 'left j...' at 0x2202E255BE8>, <Identifier 'bb' at 0x2202E25A480>, <Keyword 'on' at 0x2202E255D68>, <Comparison 'aa.aid...' at 0x2202E25A4F8>, <Where 'where ...' at 0x2202E23FB10>, <Keyword 'group' at 0x2202E257528>, <Keyword 'by' at 0x2202E2575E8>, <Identifier 'aa.aid' at 0x2202E23FCF0>, <Keyword 'order' at 0x2202E257828>, <Keyword 'by' at 0x2202E2578E8>, <Identifier 'aa.aid' at 0x2202E23FD68>]


{'asum': 'sum(a) asum', 'bmin': 'min(b) bmin'}

In [13]:
select_q1[8]
# select_q[3].ttype

<Where 'where ...' at 0x2202E23FB10>

In [14]:
class select_query_object:
    def __init__(columns,source_tables,where,group_by,order_by,limit)

SyntaxError: invalid syntax (<ipython-input-14-234b29ad012d>, line 2)